# Protein-ligand-water systems with Interchange

<details>
    <summary><small>▼ Click here for dependency installation instructions</small></summary>
    The simplest way to install dependencies is to use the Interchange examples environment. From the root of the cloned openff-interchange repository:
    
    conda env create --name interchange-examples --file devtools/conda-envs/examples_env.yaml 
    conda activate interchange-examples
    pip install -e .
    cd examples/protein_ligand/
    jupyter notebook protein_ligand.ipynb
    
</details>

In this example, we'll take a docked protein-ligand system from an OpenFF benchmark data set, parameterize and solvate it, and export the parameterized system to a variety of simulation engines.

<div class="alert alert-danger" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    <b>🚧 This code is not production-ready</b><br />
    This example describes a number of procedures that are buggy, poorly performing, or outright broken. Get excited about what's coming, but hold off on using this in production work.
</div>

In [ ]:
import urllib

import mdtraj as md
import numpy as np
from openff.evaluator.utils.packmol import pack_box
from openff.toolkit import ForceField, Molecule
from openff.units import unit

from openff.interchange import Interchange
from openff.interchange.drivers import (
    get_amber_energies,
    get_gromacs_energies,
    get_openmm_energies,
)
from openff.interchange.drivers.all import get_summary_data

## Collecting structures

In this example we'll use starting coordinates data from [MCL1], which is part of the [Protein Ligand Benchmark] data set curated by the Open Force Field Initiative. Conveniently for the purposes of this example, the ligand is already docked and the protein is relatively small (~2000 atoms), so we can focus on using Interchange without too much prep.

[Protein Ligand Benchmark]: https://github.com/openforcefield/protein-ligand-benchmark#proteinligandbenchmarks
[MCL1]: https://github.com/openforcefield/protein-ligand-benchmark/tree/8c94c0dcc892dfd77992567294b1ff31c62e8695/plbenchmark/sample_data/2020-08-26_mcl1_sample

Start by downloading the protein and ligand structure files from GitHub:

In [ ]:
url = (
    "https://raw.githubusercontent.com/openforcefield/protein-ligand-benchmark/"
    "8c94c0dcc892dfd77992567294b1ff31c62e8695/plbenchmark/sample_data/2020-08-26_mcl1_sample/"
)

urllib.request.urlretrieve(url + "01_protein/crd/protein.pdb", "protein.pdb")
urllib.request.urlretrieve(url + "02_ligands/lig_23/crd/lig_23.sdf", "lig_23.sdf")

# `protein.pdb` and `lig_23.sdf` should be in the local path now
!ls -lhrt

The PDB file includes a few waters; the OpenFF Toolkit currently does not support parsing multi-component PDB files, so we'll use [MDTraj](https://mdtraj.org/) to parse the file and save just the protein to a new file.

In [ ]:
protein_with_waters = md.load("protein.pdb")
protein_pdb = protein_with_waters.atom_slice(
    protein_with_waters.top.select("chainid 0")
)
protein_pdb.save("sliced.pdb")

## Preparing components

This system has three components: Protein, ligand, and solvent (water). For each component, we need positions and parameters. Our protein and ligand positions come from PDBs, and we'll generate solvent coordinates ourselves. For parameters, the Sage force field will be perfect for the ligand and water, but doesn't support proteins - they're coming in Rosemary. In the meantime, we'll use a recent Amber force field, which has a compatible charge generation scheme and functional form.

Unfortunately, this means we need to treat each component seperately. Fortunately, Interchange provides a great way to combine separate systems, which we'll see in a bit.

### Protein component

Let's start with the protein component. The `Molecule.from_polymer_pdb()` method constructs a `Molecule` from a PDB file encoding a protein. A `Molecule` object stores a molecule's chemical identity, bond graph, and co-ordinates. The OpenFF Toolkit doesn't accept PDB files for small molecules because they  don't have enough chemical information, but it makes an exception for biopolymers like proteins. No-one wants to write up a SMILES string for an entire protein! 

<div class="alert alert-danger" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    <b>🚧 This code is not production-ready</b><br />
    The <code>Molecule.from_polymer_pdb()</code> has some limitations. If you encounter issues using it or seek additional functionality, please report to the OpenFF Toolkit <a href=f"https://github.com/openforcefield/openff-toolkit/issues/new/choose">issue tracker</a>.
</div>

In [ ]:
protein = Molecule.from_polymer_pdb("sliced.pdb")

We can visualize the protein to make sure its reasonable:

In [ ]:
protein.visualize(backend="nglview")

OpenFF maintains a [port](https://github.com/openforcefield/openff-amber-ff-ports) of Amber ff14sb, which we'll use for the protein parameters. We're using the `impropers` variant because Interchange doesn't support Amber's improper torsion function.


<div class="alert alert-danger" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    <b>🚧 This code is not production-ready</b><br />
    The Amber ff14sb port is intended as a proof-of-concept for SMIRNOFF protein force fields. It does not precisely match the energetics or forces of the original ff14sb, and Interchange is missing features required for it to work correctly. Wait for protein support in the Rosemary force field to use this in production.
</div>

In [ ]:
ff14sb = ForceField("ff14sb_off_impropers_0.0.3.offxml")

We can use the `Interchange.from_smirnoff` constructor method to combine the `protein` molecule (with coordinates) and the `ff14sb` force field into an `Interchange`.

In [ ]:
protein_intrcg = Interchange.from_smirnoff(
    force_field=ff14sb,
    topology=protein.to_topology(),
)

### Ligand component

SDF files encode all the chemical information the Toolkit needs to construct a `Molecule`, so we can use the general-purpose small molecule `from_file` method for the ligand:

In [ ]:
ligand = Molecule.from_file("lig_23.sdf")

We'll use the [OpenFF 2.0.0 "Sage"] force field for the ligand, which is a production-ready small molecule [SMIRNOFF] force field. Its coordinates are taken from the `lig_23.sdf` file we downloaded earlier. We just want to do some point energy calculations as a proof of concept, so we'll use the unconstrained variant of Sage (see the [FAQ] for details).

[OpenFF 2.0.0 "Sage"]: https://openforcefield.org/community/news/general/sage2.0.0-release/
[FAQ]: https://open-forcefield-toolkit.readthedocs.io/en/stable/faq.html#what-does-unconstrained-mean-in-a-force-field-name
[SMIRNOFF]: https://open-forcefield-toolkit.readthedocs.io/en/stable/users/smirnoff.html

In [ ]:
ligand_intrcg = Interchange.from_smirnoff(
    force_field=ForceField("openff_unconstrained-2.0.0.offxml"),
    topology=[ligand],
)

Now that we have two interchanges, we can combine them with the `+` operator! We'll need a combined system to solvate too, so this'll be useful in a second.

<div class="alert alert-danger" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    <b>🚧 This code is not production-ready</b><br />
    The <code>+</code> operator is unstable and may break unexpectedly or be removed altogether at any time. In the future, OpenFF force fields will support biopolymers and combining Interchanges will be less necessary than at present. Don't combine force fields in production code.
</div>

In [ ]:
docked_intrcg = protein_intrcg + ligand_intrcg

In addition to making it easy to parameterize systems for all sorts of engines, Interchange makes it easy to visualize systems. We can use the `visualize()` method to view our docked system in NGLView and make sure the co-ordinates make sense:

In [ ]:
w = docked_intrcg.visualize()
w.clear_representations()
w.add_representation(
    "licorice",
    radius=0.1,
    selection=[*range(protein_intrcg.topology.n_atoms)],
)
w.add_representation(
    "spacefill",
    selection=[*range(protein_intrcg.topology.n_atoms, docked_intrcg.topology.n_atoms)],
)
w

### Solvent component

We'll reuse the Sage force field from earlier here, as it includes parameters for TIP3P water, but first we need coordinates for our solvated system. This is still a pain point in the OpenFF ecosystem, but we can use some tools from [OpenFF Evaluator] to smooth things over a little. We're adding a fixed amount of water for this quick example so the density will be wrong, but imagine it's right:

[OpenFF Evaluator]: https://github.com/openforcefield/openff-evaluator

In [ ]:
# Construct a water molecule
water = Molecule.from_smiles("O")
water.generate_conformers(n_conformers=1)

# Come up with a box size based on the size of the protein plus a 2 nm buffer
xyz = protein.conformers[0]
centroid = xyz.sum(axis=0) / xyz.shape[0]
protein_radius = np.sqrt(((xyz - centroid) ** 2).sum(axis=-1).max())
box_size = np.ones(3) * protein_radius * 2 + 2 * unit.nanometer

# pack_box needs a file to read a solute structure from
docked_intrcg.to_pdb("docked.pdb")

# Pack the box with water
n_water = 1000
packed_trj, _ = pack_box(
    molecules=[water],
    number_of_copies=[n_water],
    structure_to_solvate="docked.pdb",
    box_size=box_size,
)

# Get the values we need out of the trajectory object
xyz = packed_trj.xyz[0] * unit.nanometer
box = packed_trj.unitcell_vectors[0] * unit.nanometer

And now we can create the interchange! The `xyz` array from `pack_box` includes the positions of the solute, so we'll apply it to the combined interchange in a bit.

In [ ]:
water_intrcg = Interchange.from_smirnoff(
    force_field=ForceField("openff_unconstrained-2.0.0.offxml"),
    topology=[water] * n_water,
)

## Putting the system together

Now that we've got all the pieces, we can combine the docked protein-ligand system with the solvent, and add in the positions and box vectors we just worked out

<div class="alert alert-danger" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    <b>🚧 This code is not production-ready</b><br />
    The <code>+</code> operator is unstable and under-tested and may break unexpectedly or be removed altogether.
</div>

In [ ]:
system_intrcg = docked_intrcg + water_intrcg
system_intrcg.positions = xyz
system_intrcg.box = box

In [ ]:
w = system_intrcg.visualize()
w.clear_representations()
# Protein rep
w.add_representation(
    "licorice",
    radius=0.2,
    selection=[*range(protein_intrcg.topology.n_atoms)],
)
# Ligand rep
w.add_representation(
    "spacefill",
    selection=[*range(protein_intrcg.topology.n_atoms, docked_intrcg.topology.n_atoms)],
)
# Water rep
w.add_representation(
    "licorice",
    radius=0.1,
    selection=[*range(docked_intrcg.topology.n_atoms, system_intrcg.topology.n_atoms)],
)
w

## Exporting to simulation engines

Finally, we can export the final Interchange object to models understood by various simulation engines. Some of these exports are not yet optimized for large files.

### OpenMM

In [ ]:
openmm_system = system_intrcg.to_openmm()
openmm_topology = system_intrcg.topology.to_openmm(ensure_unique_atom_names=False)
print(type(openmm_system), type(openmm_topology))

### Amber

In [ ]:
system_intrcg.to_inpcrd("out.inpcrd")
system_intrcg.to_prmtop("out.prmtop")

### LAMMPS

In [ ]:
# LAMMPS support is temporarily offline
if False:
    system_intrcg.to_lammps("out.lmp")

### GROMACS

Interchange's GROMACS support is still a work in progress and a little slow; these will be faster in a future release.

In [ ]:
if False:
    system_intrcg.to_gro("out.gro")
    system_intrcg.to_top("out.top")

## Energy tests

In order to verify the accuracy of each export, we can use functions in the `drivers` module to call out to each engine to evaluate single-point energies. Under the hood, each function uses the export functions just as we did in the above cells. The GROMACS and Amber exports are a little slower than the OpenMM export, so this cell might take a minute to execute.

In [ ]:
print("OpenMM " + str(get_openmm_energies(system_intrcg)))
print("Amber " + str(get_amber_energies(system_intrcg)))

Since some of the export functions are not yet performant for systems of this size, here we only evaluate the OpenMM, and Amber interfaces. Efficient support for LAMMPS GROMACS is coming!

In the future, the function `get_summary_data` will support this sort of testing automatically and efficiently. As a sneak peek, below is the result of calling that function on an `Interchange` that contains only the ligand. The data are presented as a Pandas DataFrame, which includes convenient methods for summary statistics.

In [ ]:
ligand_intrcg.box = box
summary = get_summary_data(ligand_intrcg)
summary

In [ ]:
summary.describe()